In [16]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [17]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [18]:
subject = 'Amazonas - Consumo de Cimento (t)'
split_index = 12 #Referente aos 13 anos de input  

In [19]:
data = pd.read_csv('../Inputs Atuais/2008_01_model_input_AM.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data = data.drop('Amazonas - Produção de Cimento (t)', axis=1)
data

,Ano,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),...,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda,Amazonas - Consumo de Cimento (t)
0,2008,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,0.760020,0.703356,5.483335e+08,2.546644e+09,1.874883e+10,...,29.241577,1002.673957,1407.801557,29.368351,23.012008,70.408467,1.658576,0.678960,0.674670,769.233000
1,2009,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,0.761567,0.704503,5.302764e+08,2.679945e+09,2.055015e+10,...,30.906682,1002.901517,1393.526887,28.853124,22.769544,71.758162,1.685334,0.682960,0.672987,744.252000
2,2010,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,0.762693,0.705545,5.046425e+08,2.891930e+09,2.342247e+10,...,35.544249,1003.401682,1311.661141,28.071885,22.643630,74.482804,1.537130,0.686918,0.671387,796.116000
3,2011,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,0.763406,0.706485,4.854622e+08,3.091988e+09,2.604265e+10,...,39.206921,1003.835072,1276.962232,27.828830,22.427653,74.624578,1.468307,0.690831,0.669868,948.011000
4,2012,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,0.763641,0.707029,4.867661e+08,3.189506e+09,2.708753e+10,...,40.620066,1004.045485,1269.809720,27.699098,22.391859,74.970536,1.457821,0.694062,0.668360,994.932000
5,2013,5.434368e+07,2.507099e+06,12.394100,4.415452e+07,0.763611,0.708365,5.225848e+08,3.093872e+09,2.523394e+10,...,39.498004,1004.283223,1266.017495,27.731222,22.404779,74.742115,1.485513,0.699234,0.667140,1036.606000
6,2014,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.763162,0.708895,5.676413e+08,2.779340e+09,1.954361e+10,...,38.217511,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,0.702586,0.665819,771.567857
7,2015,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.762342,0.709137,5.179392e+08,2.561731e+09,1.327596e+10,...,38.739113,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,0.705552,0.664387,746.867486
8,2016,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.770069,0.713039,4.524144e+08,2.516859e+09,1.011370e+10,...,39.689203,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,0.716221,0.658519,739.457657
9,2017,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.769075,0.713224,4.213689e+08,2.497710e+09,8.456309e+09,...,39.521434,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,0.717241,0.658930,687.211000


In [20]:
input_data = data.iloc[:-2, 1:-1]
input_data

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,5.017229e+07,2.568805e+06,12.460706,4.349579e+07,0.760020,0.703356,5.483335e+08,2.546644e+09,1.874883e+10,2.351319e+10,...,8.396886,29.241577,1002.673957,1407.801557,29.368351,23.012008,70.408467,1.658576,0.678960,0.674670
1,5.047410e+07,2.574930e+06,12.433926,4.345535e+07,0.761567,0.704503,5.302764e+08,2.679945e+09,2.055015e+10,2.446518e+10,...,8.420659,30.906682,1002.901517,1393.526887,28.853124,22.769544,71.758162,1.685334,0.682960,0.672987
2,5.169307e+07,2.569006e+06,12.457785,4.379523e+07,0.762693,0.705545,5.046425e+08,2.891930e+09,2.342247e+10,2.606035e+10,...,8.389077,35.544249,1003.401682,1311.661141,28.071885,22.643630,74.482804,1.537130,0.686918,0.671387
3,5.272696e+07,2.545977e+06,12.460974,4.398860e+07,0.763406,0.706485,4.854622e+08,3.091988e+09,2.604265e+10,2.845738e+10,...,8.376520,39.206921,1003.835072,1276.962232,27.828830,22.427653,74.624578,1.468307,0.690831,0.669868
4,5.353767e+07,2.524602e+06,12.414889,4.403561e+07,0.763641,0.707029,4.867661e+08,3.189506e+09,2.708753e+10,3.181492e+10,...,11.321769,40.620066,1004.045485,1269.809720,27.699098,22.391859,74.970536,1.457821,0.694062,0.668360
5,5.434368e+07,2.507099e+06,12.394100,4.415452e+07,0.763611,0.708365,5.225848e+08,3.093872e+09,2.523394e+10,3.629162e+10,...,12.008921,39.498004,1004.283223,1266.017495,27.731222,22.404779,74.742115,1.485513,0.699234,0.667140
6,5.487107e+07,2.472487e+06,12.359116,4.412824e+07,0.763162,0.708895,5.676413e+08,2.779340e+09,1.954361e+10,4.143397e+10,...,12.610217,38.217511,1004.431965,1277.324904,28.055466,22.412378,73.347472,1.524622,0.702586,0.665819
7,5.527657e+07,2.446996e+06,12.336147,4.411441e+07,0.762342,0.709137,5.179392e+08,2.561731e+09,1.327596e+10,4.469706e+10,...,13.139356,38.739113,1004.517645,1275.202120,28.150076,22.434009,72.952065,1.509930,0.705552,0.664387
8,5.561997e+07,2.436680e+06,12.339032,4.416689e+07,0.770069,0.713039,4.524144e+08,2.516859e+09,1.011370e+10,4.640544e+10,...,13.263561,39.689203,1004.590070,1250.291696,27.936211,22.456139,73.785027,1.450843,0.716221,0.658519
9,5.581151e+07,2.432227e+06,12.358279,4.424802e+07,0.769075,0.713224,4.213689e+08,2.497710e+09,8.456309e+09,4.607888e+10,...,12.821072,39.521434,1004.691076,1227.047962,27.873459,22.397053,73.760599,1.416718,0.717241,0.658930


In [21]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      744.252000
1      796.116000
2      948.011000
3      994.932000
4     1036.606000
5      771.567857
6      746.867486
7      739.457657
8      687.211000
9      350.177000
10     533.444000
11     595.320000
12     610.155000
13     640.274000
14            NaN
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [22]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,-1.823984,1.303637,1.392423,-1.881765,-1.489637,-1.645646,1.204752,-0.990234,0.166091,-1.487840,...,-1.373236,-2.227879,-2.010367,1.924851,2.625774,2.633383,-2.467290,1.708024,-1.621756,1.535111
1,-1.673536,1.416732,0.791547,-2.028261,-0.964358,-1.283213,0.820657,-0.546688,0.428911,-1.378556,...,-1.361444,-1.801387,-1.684250,1.716762,1.560169,1.369363,-1.408300,1.984284,-1.320949,1.248581
2,-1.065906,1.307356,1.326890,-0.796982,-0.582138,-0.954158,0.275392,0.158672,0.847995,-1.195437,...,-1.377109,-0.613544,-0.967461,0.523363,-0.055613,0.712946,0.729492,0.454191,-1.023245,0.976441
3,-0.550533,0.882147,1.398444,-0.096457,-0.340162,-0.657254,-0.132596,0.824346,1.230291,-0.920269,...,-1.383337,0.324594,-0.346368,0.017539,-0.558307,-0.412991,0.840731,-0.256354,-0.728908,0.717851
4,-0.146413,0.487481,0.364414,0.073831,-0.260467,-0.485225,-0.104860,1.148829,1.382743,-0.534838,...,0.077489,0.686550,-0.044825,-0.086726,-0.826622,-0.599598,1.112174,-0.364616,-0.485905,0.461287
5,0.255366,0.164315,-0.102045,0.504621,-0.270630,-0.063266,0.657046,0.830615,1.112297,-0.020933,...,0.418312,0.399151,0.295877,-0.142008,-0.760182,-0.532243,0.932952,-0.078717,-0.096908,0.253666
6,0.518259,-0.474755,-0.886983,0.409399,-0.423086,0.104286,1.615453,-0.215960,0.282052,0.569384,...,0.716551,0.071172,0.509040,0.022826,-0.089571,-0.492626,-0.161305,0.325049,0.155273,0.028744
7,0.720395,-0.945418,-1.402348,0.359293,-0.701403,0.180503,0.558230,-0.940033,-0.632425,0.943972,...,0.979001,0.204772,0.631829,-0.008119,0.106103,-0.379858,-0.471547,0.173370,0.378325,-0.214922
8,0.891571,-1.135894,-1.337608,0.549425,1.921611,1.413182,-0.835563,-1.089341,-1.093812,1.140087,...,1.040606,0.448123,0.735622,-0.371251,-0.336219,-0.264487,0.182007,-0.436659,1.180811,-1.213618
9,0.987051,-1.218110,-0.905767,0.843354,1.584260,1.471729,-1.495939,-1.153056,-1.335633,1.102599,...,0.821134,0.405152,0.880374,-0.710087,-0.466003,-0.572517,0.162840,-0.788968,1.257539,-1.143706


In [23]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      744.252000
1      796.116000
2      948.011000
3      994.932000
4     1036.606000
5      771.567857
6      746.867486
7      739.457657
8      687.211000
9      350.177000
10     533.444000
11     595.320000
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [24]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
12,0.548425,-0.389186,-0.34651,0.585399,-0.362055,0.429222,1.108405,2.635823,-0.189959,0.726405,...,0.598328,0.774435,0.860523,-1.989017,-1.013503,-0.547354,1.083075,-1.613773,0.941105,-1.177883


In [25]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:-2]
test_target

12    610.155
Name: Amazonas - Consumo de Cimento (t), dtype: float64

In [40]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-1:], train_target.iloc[-1:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [41]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")

    return winner_seed

In [42]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[528636567, 2045946593, 4093479462, 3368548987, 321891767, 1524243301, 2161721358, 3001585908, 3565685097, 3429680123, 3203105547, 2688975490, 3793756765, 2007400667, 877033915, 3180986421, 2034344264, 127255383, 4047828273, 1683327739, 2867694121, 481822182, 1521215851, 551639582, 3056386127, 2855850882, 584366009, 2495796224, 2793992749, 1647395079, 490531977, 2553801391, 2156412941, 2773769722, 1538261621, 2996018419, 860514536, 909597401, 1046739974, 3204786689, 1752399703, 2377718909, 548424455, 4140947610, 2108145290, 4234472802, 2783369451, 3402974386, 1948528126, 3125960706, 2122818996, 3705765979, 3847551660, 2222700287, 1416668566, 1825638658, 535747317, 4288433768, 1317282447, 4115726973, 2202085542, 1766178834, 1075751304, 2388056524, 1051996119, 4081650774, 3283388623, 2316599772, 3504495154, 2997364408, 573511194, 1633773686, 3463747727, 835584206, 3852395144, 451209963, 155181663, 2071576042, 2636078506, 2036034175, 698804846, 1593013056, 2647147718, 1191001605, 30931072

val_loss: 0.5359948873519897


Step: 84 ___________________________________________
val_loss: 21.203035354614258


Step: 85 ___________________________________________
val_loss: 1.0578192472457886


Step: 86 ___________________________________________
val_loss: 0.0007917024195194244


Step: 87 ___________________________________________
val_loss: 0.042207375168800354


Step: 88 ___________________________________________
val_loss: 0.5631410479545593


Step: 89 ___________________________________________
val_loss: 1328.9014892578125


Step: 90 ___________________________________________
val_loss: 0.3900911211967468


Step: 91 ___________________________________________
val_loss: 0.002414081245660782


Step: 92 ___________________________________________
val_loss: 73.5318832397461


Step: 93 ___________________________________________
val_loss: 1.341104507446289e-05


Step: 94 ___________________________________________
val_loss: 1.0464340448379517e-05


Step: 95 ________________________

In [43]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 458ms/step - loss: 590084.0625 - val_loss: 323621.8750
Epoch 2/10000
1/1 [==============================] - 0s 13ms/step - loss: 569479.4375 - val_loss: 159418.1562
Epoch 3/10000
1/1 [==============================] - 0s 13ms/step - loss: 645393.6875 - val_loss: 586113.4375
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 549785.5000 - val_loss: 458552.5000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 532673.3125 - val_loss: 495397.4062
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 518017.6250 - val_loss: 468582.5312
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 502739.0000 - val_loss: 244717.0469
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 462024.2500 - val_loss: 511248.4062
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 452987.1250 - val_loss: 488412.7812
Epoch 10/

Epoch 75/10000
1/1 [==============================] - 0s 14ms/step - loss: 95363.3047 - val_loss: 155491.5156
Epoch 76/10000
1/1 [==============================] - 0s 13ms/step - loss: 93216.2188 - val_loss: 151484.0625
Epoch 77/10000
1/1 [==============================] - 0s 13ms/step - loss: 93831.4766 - val_loss: 142543.0000
Epoch 78/10000
1/1 [==============================] - 0s 13ms/step - loss: 92362.2734 - val_loss: 142944.4375
Epoch 79/10000
1/1 [==============================] - 0s 14ms/step - loss: 89613.5312 - val_loss: 139667.8281
Epoch 80/10000
1/1 [==============================] - 0s 13ms/step - loss: 92881.6953 - val_loss: 127787.1797
Epoch 81/10000
1/1 [==============================] - 0s 13ms/step - loss: 83435.1797 - val_loss: 127104.4141
Epoch 82/10000
1/1 [==============================] - 0s 13ms/step - loss: 83477.3359 - val_loss: 123069.0859
Epoch 83/10000
1/1 [==============================] - 0s 13ms/step - loss: 90049.1953 - val_loss: 131471.3281
Epoch 84/1

1/1 [==============================] - 0s 13ms/step - loss: 25746.6250 - val_loss: 31674.6348
Epoch 150/10000
1/1 [==============================] - 0s 13ms/step - loss: 34796.3984 - val_loss: 27586.1191
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 38864.3555 - val_loss: 70444.0938
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 37351.9492 - val_loss: 55764.8594
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 36048.0195 - val_loss: 57271.3203
Epoch 154/10000
1/1 [==============================] - 0s 13ms/step - loss: 33358.6055 - val_loss: 55329.4531
Epoch 155/10000
1/1 [==============================] - 0s 14ms/step - loss: 41846.0195 - val_loss: 18434.6582
Epoch 156/10000
1/1 [==============================] - 0s 13ms/step - loss: 40479.1133 - val_loss: 53645.8984
Epoch 157/10000
1/1 [==============================] - 0s 14ms/step - loss: 34233.4648 - val_loss: 43801.1836
Epoch 158/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 17388.3750 - val_loss: 16420.0508
Epoch 224/10000
1/1 [==============================] - 0s 14ms/step - loss: 17410.0527 - val_loss: 16572.5703
Epoch 225/10000
1/1 [==============================] - 0s 14ms/step - loss: 17365.4512 - val_loss: 15738.8545
Epoch 226/10000
1/1 [==============================] - 0s 13ms/step - loss: 17343.9512 - val_loss: 16200.4385
Epoch 227/10000
1/1 [==============================] - 0s 13ms/step - loss: 16626.5879 - val_loss: 15869.3096
Epoch 228/10000
1/1 [==============================] - 0s 13ms/step - loss: 17927.1953 - val_loss: 12323.7510
Epoch 229/10000
1/1 [==============================] - 0s 13ms/step - loss: 17231.6328 - val_loss: 14914.1582
Epoch 230/10000
1/1 [==============================] - 0s 13ms/step - loss: 16200.0586 - val_loss: 14822.0068
Epoch 231/10000
1/1 [==============================] - 0s 13ms/step - loss: 15925.3154 - val_loss: 13825.9502
Epoch 232/10000
1/1 [=====

1/1 [==============================] - 0s 13ms/step - loss: 12427.6572 - val_loss: 2843.2705
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 12206.1797 - val_loss: 2496.2600
Epoch 300/10000
1/1 [==============================] - 0s 13ms/step - loss: 11601.0479 - val_loss: 3023.1003
Epoch 301/10000
1/1 [==============================] - 0s 14ms/step - loss: 12525.5029 - val_loss: 2051.6355
Epoch 302/10000
1/1 [==============================] - 0s 13ms/step - loss: 12154.2969 - val_loss: 2478.5193
Epoch 303/10000
1/1 [==============================] - 0s 14ms/step - loss: 13818.0908 - val_loss: 897.5005
Epoch 304/10000
1/1 [==============================] - 0s 13ms/step - loss: 12801.6689 - val_loss: 2017.3992
Epoch 305/10000
1/1 [==============================] - 0s 13ms/step - loss: 12043.6572 - val_loss: 2339.3318
Epoch 306/10000
1/1 [==============================] - 0s 14ms/step - loss: 12805.4375 - val_loss: 1287.8566
Epoch 307/10000
1/1 [===============

...

1/1 [==============================] - 0s 13ms/step - loss: 11575.0156 - val_loss: 45.7118
Epoch 1579/10000
1/1 [==============================] - 0s 13ms/step - loss: 11259.7031 - val_loss: 530.9728
Epoch 1580/10000
1/1 [==============================] - 0s 13ms/step - loss: 11043.1240 - val_loss: 138.6843
Epoch 1581/10000
1/1 [==============================] - 0s 13ms/step - loss: 10975.5547 - val_loss: 245.0226
Epoch 1582/10000
1/1 [==============================] - 0s 13ms/step - loss: 10960.5361 - val_loss: 214.9077
Epoch 1583/10000
1/1 [==============================] - 0s 14ms/step - loss: 10953.8252 - val_loss: 223.0505
Epoch 1584/10000
1/1 [==============================] - 0s 14ms/step - loss: 11093.2588 - val_loss: 448.6619
Epoch 1585/10000
1/1 [==============================] - 0s 14ms/step - loss: 12361.5547 - val_loss: 190.9487
Epoch 1586/10000
1/1 [==============================] - 0s 13ms/step - loss: 10969.2695 - val_loss: 377.9018
Epoch 1587/10000
1/1 [===============

1/1 [==============================] - 0s 13ms/step - loss: 14144.9209 - val_loss: 213.7692
Epoch 1654/10000
1/1 [==============================] - 0s 13ms/step - loss: 14093.8232 - val_loss: 218.0742
Epoch 1655/10000
1/1 [==============================] - 0s 14ms/step - loss: 14044.0039 - val_loss: 222.2218
Epoch 1656/10000
1/1 [==============================] - 0s 14ms/step - loss: 13995.4346 - val_loss: 226.4801
Epoch 1657/10000
1/1 [==============================] - 0s 14ms/step - loss: 13948.0771 - val_loss: 230.7807
Epoch 1658/10000
1/1 [==============================] - 0s 14ms/step - loss: 13659.6123 - val_loss: 295.7732
Epoch 1659/10000
1/1 [==============================] - 0s 13ms/step - loss: 13863.4521 - val_loss: 224.1951
Epoch 1660/10000
1/1 [==============================] - 0s 14ms/step - loss: 15584.2734 - val_loss: 624.6674
Epoch 1661/10000
1/1 [==============================] - 0s 14ms/step - loss: 13876.3945 - val_loss: 170.4153
Epoch 1662/10000
1/1 [==============

1/1 [==============================] - 0s 14ms/step - loss: 12143.7148 - val_loss: 793.0597
Epoch 1729/10000
1/1 [==============================] - 0s 14ms/step - loss: 12044.0342 - val_loss: 972.2567
Epoch 1730/10000
1/1 [==============================] - 0s 14ms/step - loss: 11936.0928 - val_loss: 1124.9384
Epoch 1731/10000
1/1 [==============================] - 0s 14ms/step - loss: 12008.2939 - val_loss: 960.6897
Epoch 1732/10000
1/1 [==============================] - 0s 14ms/step - loss: 11930.3525 - val_loss: 1154.3280
Epoch 1733/10000
1/1 [==============================] - 0s 14ms/step - loss: 15956.6006 - val_loss: 2329.6353
Epoch 1734/10000
1/1 [==============================] - 0s 13ms/step - loss: 12577.1045 - val_loss: 593.0372
Epoch 1735/10000
1/1 [==============================] - 0s 13ms/step - loss: 12043.7812 - val_loss: 1146.2506
Epoch 1736/10000
1/1 [==============================] - 0s 14ms/step - loss: 12227.0557 - val_loss: 810.6609
Epoch 1737/10000
1/1 [==========

Epoch 1803/10000
1/1 [==============================] - 0s 13ms/step - loss: 14332.4717 - val_loss: 2981.4172
Epoch 1804/10000
1/1 [==============================] - 0s 14ms/step - loss: 11974.3740 - val_loss: 1420.5409
Epoch 1805/10000
1/1 [==============================] - 0s 13ms/step - loss: 11840.4766 - val_loss: 1517.1700
Epoch 1806/10000
1/1 [==============================] - 0s 13ms/step - loss: 11834.8555 - val_loss: 1508.0262
Epoch 1807/10000
1/1 [==============================] - 0s 13ms/step - loss: 11833.2295 - val_loss: 1520.0718
Epoch 1808/10000
1/1 [==============================] - 0s 14ms/step - loss: 11831.6533 - val_loss: 1526.5465
Epoch 1809/10000
1/1 [==============================] - 0s 13ms/step - loss: 11794.9033 - val_loss: 1788.8966
Epoch 1810/10000
1/1 [==============================] - 0s 13ms/step - loss: 11838.8799 - val_loss: 1477.0708
Epoch 1811/10000
1/1 [==============================] - 0s 13ms/step - loss: 11828.2764 - val_loss: 1564.9934
Epoch 1812

1/1 [==============================] - 0s 14ms/step - loss: 11738.6172 - val_loss: 2140.5137
Epoch 1878/10000
1/1 [==============================] - 0s 13ms/step - loss: 14165.8779 - val_loss: 3452.4160
Epoch 1879/10000
1/1 [==============================] - 0s 13ms/step - loss: 11895.3906 - val_loss: 1840.6058
Epoch 1880/10000
1/1 [==============================] - 0s 13ms/step - loss: 11747.5078 - val_loss: 2214.2297
Epoch 1881/10000
1/1 [==============================] - 0s 13ms/step - loss: 11736.9404 - val_loss: 2120.1179
Epoch 1882/10000
1/1 [==============================] - 0s 14ms/step - loss: 11735.5391 - val_loss: 2149.8369
Epoch 1883/10000
1/1 [==============================] - 0s 13ms/step - loss: 11734.7686 - val_loss: 2147.7488
Epoch 1884/10000
1/1 [==============================] - 0s 13ms/step - loss: 11734.0596 - val_loss: 2153.8743
Epoch 1885/10000
1/1 [==============================] - 0s 13ms/step - loss: 11733.3740 - val_loss: 2157.8530
Epoch 1886/10000
1/1 [=====

In [44]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 35ms/step
[610.155](test_target) - [[595.3222]](prediction) = 14.832795410156223


In [45]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [46]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 4, winner_seed)
predictions

train_input:


,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
0,-0.924122,-0.742263,0.825349,-0.569521,-1.283796,-1.243968,1.148205,-1.122043,-1.121418,-1.109924,...,-0.396211,-0.995947,-1.047594,0.868878,1.133056,1.331872,-1.066913,0.489423,-1.226856,1.235124
1,-0.465030,1.413621,-1.407209,-0.836281,0.128184,0.039396,0.140893,-0.184474,-0.185492,-0.204018,...,1.373802,-0.371548,-0.298948,0.531888,0.166368,-0.254125,-0.270454,0.904353,0.004233,-0.021028
2,1.389153,-0.671358,0.581860,1.405802,1.155611,1.204571,-1.289098,1.306518,1.306910,1.313942,...,-0.977591,1.367495,1.346542,-1.400766,-1.299425,-1.077747,1.337367,-1.393776,1.222623,-1.214095


train_target:


,Amazonas - Consumo de Cimento (t)
0,744.252
1,796.116
2,948.011


test_input:


...

,Amazonas - PIB - Estadual,Amazonas - PIB - Construção Civil,Amazonas - PIB - Per Capita,Amazonas - PIB - Preços de Mercado,Amazonas - IDH Longevidade,Amazonas - IDH,Contratação Comercial (Bi R$),Contratação Home equity (Bi R$),Contratação Livre (Bi R$),Contratação FGTS (Bi R$),...,Amazonas - Desemprego,Precipitation (mm/day),Atmospheric Pressure(mB),Global Radiation - Daylight (Kj/m²),Air Temperature - Dry Bulb (°C),Dew Point Temperature (°C),Air Relative Humidity (%),Wind Gust (m/s),Amazonas - IDH Educacao,Amazonas - IDH Renda
12,0.548425,-0.389186,-0.34651,0.585399,-0.362055,0.429222,1.108405,2.635823,-0.189959,0.726405,...,0.598328,0.774435,0.860523,-1.989017,-1.013503,-0.547354,1.083075,-1.613773,0.941105,-1.177883


test_target:


,Amazonas - Consumo de Cimento (t)
12,610.155


1/1 [==============================] - 0s 35ms/step
Error: 14.832795410156223




[948.0066528320312,
 986.7642822265625,
 1010.6630249023438,
 774.6240844726562,
 746.8082275390625,
 738.9821166992188,
 687.2193603515625,
 596.0316162109375,
 605.9085083007812,
 595.3222045898438]

In [47]:
display(targets)
display(predictions)

[994.932,
 1036.606,
 771.5678571428572,
 746.8674857142857,
 739.4576571428571,
 687.211,
 350.177,
 533.444,
 595.32,
 610.155]

[948.0066528320312,
 986.7642822265625,
 1010.6630249023438,
 774.6240844726562,
 746.8082275390625,
 738.9821166992188,
 687.2193603515625,
 596.0316162109375,
 605.9085083007812,
 595.3222045898438]

In [48]:
mae = mean_absolute_error(predictions, targets)
mae

84.77917988281249

In [49]:
porcentage = mae/np.mean(targets)
porcentage

0.11998630558168515